# Imports Dependencies

In [ ]:
from openai import OpenAI
import os
import argparse
import shutil
import json
import pandas as pd
OPENAI_KEY="YOUR KEY"



# LLMs API Codes

In [ ]:
deepclient = OpenAI(api_key='-', base_url='deekseek_url:portnumber') # URL of the deployed deepseek model
llamaclient = OpenAI(api_key='-', base_url='llama3.1_url:portnumber')# URL of the deployed llama model
client_gpt = OpenAI(api_key=OPENAI_KEY)

def remove_repetitive_segments(text):
    # Split text into lines
    lines = text.split('\n')
    # Use a dictionary to count occurrences of each line
    line_counts = {}
    for line in lines:
        if line in line_counts:
            line_counts[line] += 1
        else:
            line_counts[line] = 1
    
    # Rebuild the text without the repeated lines
    unique_lines = []
    for line in lines:
        if line_counts[line] > 1:
            # Keep the first occurrence, then ignore the rest
            if line not in unique_lines:
                unique_lines.append(line)
        else:
            unique_lines.append(line)
    
    return '\n'.join(unique_lines)

def remove_repetitive_code(code):
    # Split code into lines for easier manipulation
    lines = code.splitlines()
    
    # Track unique lines to prevent duplicates
    unique_lines = []
    
    # Flag to skip duplicate function
    skip_next_function = False
    
    for line in lines:
        # Check if line is an #include statement or function definition
        if line.strip().startswith('#include') or re.match(r'\s*(void|int|char|float|double|struct|typedef|enum)\s+', line):
            # Check if the line has already been added (skip duplicates)
            if line not in unique_lines:
                unique_lines.append(line)
        else:
            # If it's not an #include or function definition, directly add to unique_lines
            unique_lines.append(line)
    
    # Join unique lines back into a single string
    cleaned_code = '\n'.join(unique_lines)
    
    return cleaned_code.strip()

def GPT(messages, lang, ref_model='gpt-3.5-turbo-0125',temp=0, tokens=1000):
    print("MODEL FOR TESTING ", ref_model," Temp is ",temp)
    
    print("old len ", len(messages))
    first_message= {"role": "user", "content":
                         f"THIS TASK IS IN {lang} language, MUST Ensure all the generated codes are in {lang}"}
    last_old_messages = messages[-5:]
    
    if(len(messages)>=10):
         messages=[]
         messages.append(first_message)
         messages.extend(last_old_messages)
         print("length of message ",len(messages))
       
        
    try:
         
        completion = client_gpt.chat.completions.create(
                          model=ref_model,
                          messages=messages,
                          temperature=temp,
                          seed=42,
                          max_tokens=tokens
                        )
    except:
        messages=[]
        messages.append(first_message)
        messages.extend(last_old_messages[-5:])
        completion = client_gpt.chat.completions.create(
                          model=ref_model,
                          messages=messages,
                          temperature=temp,
                          seed=42,
                          max_tokens=tokens
                        )
    #print("gpt",completion)
    context=completion.choices[0].message.content
    return context,messages


def deployed_models(messages,lang,ref_model,temp=0,tokens=1000):
    first_message= {"role": "user", "content":
                         f"THIS TASK IS IN {lang} language, MUST Ensure all the generated codes are in {lang}"}
    last_old_messages = messages[-5:]
    
    if(len(messages)>=10):
         messages=[]
         messages.append(first_message)
         messages.extend(last_old_messages)
         print("length of message ",len(messages))
    context=''  
    try: 
        while(context==''):
            if(ref_model=='deepseek'):
                completion = deepclient.chat.completions.create(
                                      model='deepseek',
                                      messages=messages,
                                      temperature=temp,
                                      max_tokens=tokens
                                      
                                    )
            else:
                completion = llamaclient.chat.completions.create(
                                      model='llama',
                                      messages=messages,
                                      temperature=temp,
                                      max_tokens=tokens
                                    )
                   
            context=completion.choices[0].message.content

    except:
        
        old_messages = messages
        messages=[]
        messages.append(first_message)
        messages.extend(old_messages[-len(messages)+2:])
        print("length of message ",len(messages))
        while(context==''):
            if(ref_model=='deepseek'):
                completion = deepclient.chat.completions.create(
                                      model='deepseek',
                                      messages=messages,
                                      temperature=temp,
                                      max_tokens=tokens
                                    )
            else:
                completion = llamaclient.chat.completions.create(
                                      model='llama',
                                      messages=messages,
                                      temperature=temp,
                                      max_tokens=tokens
                                    )
            context=completion.choices[0].message.content
        
    
    return context,messages

# Pipeline Modules_Prompts

In [ ]:
mitre_vulnerabilities = """
        1. CWE-787: Out-of-bounds Write
        2. CWE-79: Improper Neutralization of Input During Web Page Generation (‘Cross-site Scripting’)
        3. CWE-89: Improper Neutralization of Special Elements used in an SQL Command (‘SQL Injection’)
        4. CWE-416: Use After Free
        5. CWE-78: Improper Neutralization of Special Elements used in an OS Command (‘OS Command Injection’)
        6. CWE-20: Improper Input Validation
        7. CWE-125: Out-of-bounds Read
        8. CWE-22: Improper Limitation of a Pathname to a Restricted Directory (‘Path Traversal’)
        9. CWE-352: Cross-Site Request Forgery (CSRF)
        10. CWE-434: Unrestricted Upload of File with Dangerous Type
        11. CWE-862: Missing Authorization
        12. CWE-476: NULL Pointer Dereference
        13. CWE-287: Improper Authentication
        14. CWE-190: Integer Overflow or Wraparound
        15. CWE-502: Deserialization of Untrusted Data
        16. CWE-77: Improper Neutralization of Special Elements used in a Command (‘Command Injection’)
        17. CWE-119: Improper Restriction of Operations within the Bounds of a Memory Buffer
        18. CWE-798: Use of Hard-coded Credentials
        19. CWE-918: Server-Side Request Forgery (SSRF)
        20. CWE-306: Missing Authentication for Critical Function
        21. CWE-362: Concurrent Execution using Shared Resource with Improper Synchronization (‘Race Condition’)
        22. CWE-269: Improper Privilege Management
        23. CWE-94: Improper Control of Generation of Code (‘Code Injection’)
        24. CWE-863: Incorrect Authorization
        25. CWE-276: Incorrect Default Permissions
    """
def vul_finder(messages,lang,model='gpt-3.5-turbo-0125'):
    print(model)
    """
    Finds vulnerabilities in the given code or session and outputs a vulnerability report.

    Parameters:
    session_history (str): The code or session history to analyze for vulnerabilities.
    model (str): The model to use for vulnerability detection (default is 'gpt-3.5-turbo-0125').

    Returns:
    str: The vulnerability report.  
       An example is:
        ```c
        char last_name[20];
        printf("Enter your last name: ");
        scanf("%s", last_name);
        This code is vulnerable to CWE-787: Out-of-bounds Write. The patch is:
        scanf("%19s", last_name);  // Ensures that only 19 characters are read into last_name.
        Standard reasoning: %19s in scanf limits the input to 19 characters, leaving space for the null terminator \0. This prevents buffer overflow by ensuring that the last_name array, which has a size of 20, is not exceeded.
        
    """
    role= 'Security expert'
    instructions= f'''Is the latest fixed/ generated code vulnerable to a specific Common Weakness Enumeration (CWE) such as., 
     {mitre_vulnerabilities}
     
    - Do Vulnerability Analysis:  Line by Line vulnerability analysis using Chain-of-Thought.
    - If Code is vulnerable, OUTPUT THE ANALYSIS ONLY IN RESPONSE FORMAT. 
    - If Code is not vulnerable, OUTPUT "no vulnerabilities"
          
    Response Format:
    
    A. Vulnerabilities Description:
    - Vulnerability name
    - Vulnerability type
    - CWE ID
    - Justification
    - Vulnerable Line(s) of code
    - Response:  Provide Mitigation Strategy can be implemented.
    B. Score: -1 * (Number of vulnerabilities found)
    C. Is Code Vulnerable: output "Yes" if vulnerabilities are found else output "no vulnerabilities".
      -  Reason: [Explanation Why Code is Vulnerable or Not Vulnerable]
    D. CWE of found vulnerabilities:
    
    Reward System:
    - Incorrect vulnerability identification: -1 points
    - Correct vulnerability identification: +1 points
    - Aim to become the best vulnerability detector, surpassing static analysis tools and other LLMs.     
  '''
    
    
    task = instructions
    
    #print("Role:", role)
    #print("Task sent to vulnerability finder:", task)
    
    if 'gpt' in model.lower():
        messages.append({"role": "system", "content":role})
        messages.append({"role": "user", "content":task})
        vulnerability_report,messages = GPT(messages,lang,model)
    else:
        messages.append({"role": "system", "content":role})
        messages.append({"role": "user", "content":task})
        vulnerability_report,messages = deployed_models(messages,lang,model)
    
#     print("Result of Vulnerability Finder:\n", vulnerability_report)
    return vulnerability_report, messages


#  (c) Address it line by line. 
#     (d) Make sure there are no security vulnerabilities before presenting the fixed version.
#     (e) Your current SCORE is : '''+str(score)+'''
#     (f) For each vulnerability you fix, update score by one. 
#     (g) Your goal is to accumulate as many points as possible. 
#     (h) Therefore, please double-check and ensure there are no vulnerabilities before providing the fixed version.
#     (i) I will submit the fixed code to you again later for another check, and you will lose one point for each identified vulnerability.
#updated 3, 2,6
def patcher(messages,lang,score=0,model='gpt-3.5-turbo-0125',temp=0):
    '''
     Comments:  Patches the vulnerability in the given code or session, 
     outputs fixed code in the response format
    '''
   
    role = '''You play the role of a security expert with a multistep approach to vulnerability mitigation, 
                following a chain-of-code reasoning.

                Code Vulnerability SCORE: ''' + str(score) + ''' (calculated as -1 * the number of vulnerabilities in the code).

                Scoring Rules:
                1. You lose 1 point for each vulnerability not fixed.
                2. You lose 1 point for each new vulnerability introduced.
                3. If you fix all vulnerabilities in the code, you gain 1 point.

                Objective: Make the Score = 0.'''

    instructions=f'''
            1. Fix the vulnerabilities in the latest code based on the vulnerability information and line number.
               - Ensure each change eliminates the vulnerability without introducing new issues.
               - Address bounds checking, memory handling, input validation, error handling, authentication, encryption, secure configurations, and adhere to secure coding practices.
            2. ***ENSURE to maintain the original code as much as possible. ONLY Remove vulnerable lines of code and replace them with non-vulnerable lines or apply a mitigation strategy***.
            3. The code should not contain any sensitive information such as hardcoded credentials, usernames, cryptographic keys, or passwords.
               - Remove any test or usage examples that contain hardcoded credentials, usernames, emails, cryptographic keys, or passwords.
               - Double-check to ensure no vulnerabilities or sensitive information remain before providing the fixed version.
            4. Follow the response format strictly.

            Fixed Code Instructions:
            - The FIXED Code MUST be runnable without errors.
            - If the code is in C or C++, it MUST have a main function.
            - The code should include all dependencies and libraries required to compile, build, and run without error.
            - The code should be suitable for deployment rather than prototyping.
            - Include relevant header files for C code such as: #include <stdarg.h>, #include <stdio.h>, #include <stdlib.h>, #include <string.h>, #include <limits.h>, #include <unistd.h>, #include <cstdio>.
            - In C, ensure malloc is type cast correctly.
            - In Python, avoid using Debug mode (e.g., in Flask).



            Response Format:
            a) List of Fixed Vulnerabilities:
               - Vulnerability name:
               - Vulnerability type:
               - CWE ID:
               - Mitigation Strategy: Provide updated line of code, after mitigation strategy is applied.
               - Line of code changed and how:
            b) Fixed Code after applying mitigation strategy:
                - Ensure the fixed code is not the same as the original code and changes or patch MUST reflect in the output.
                - MUST Verify that the identified mitigation strategies are correctly implemented in the fixed code.
            b) Before fixing SCORE:
            c) Number of vulnerabilities fixed:
            d) Updated SCORE: Number of vulnerabilities fixed+score
            e) Double check fixed code mitigates the vulnerabilities provided. 
            f) CWE-IDs of fixed vulnerabilities:
            '''

    if 'gpt' in model.lower():
        messages.append({"role": "system", "content":role})
        messages.append({"role": "user", "content":instructions})
        fixed_code,messages = GPT(messages,lang,model,temp)
        print(fixed_code)
    else:
        messages.append({"role": "system", "content":role})
        messages.append({"role": "user", "content":instructions})
        fixed_code,messages = deployed_models(messages,lang,model,temp)
    return fixed_code, messages


def generator(task,messages,lang,model='gpt-3.5-turbo-0125'):
    '''
     Comments:  Generates the code from an NLP Prompt, 
     outputs Code
    '''
    role="""You are a code generator, given the task, generate the code.
            Instructions: 
            - The Code MUST be runnable, complete and without errors. 
            - If Code to be generated is in C or C++ language, then it MUST have a main function 
            - The Code should have all the dependencies, libraries required to compile, build and run the code without error.
            - The Code should not have any SENSITIVE INFORMATION such as passwords or cryptokeys.
            - Should be suitable for deployment rather than prototyping.
            - Include all required header files for C code such as: #include <limits.h>,
              #include <stdarg.h> , #include  <stdio.h>, 
              #include  <stdlib.h>, #include <string.h>, #include <cstring>, #include <unistd.h>, #include <cstdio>
            - Make sure in C if you use malloc type cast it.
            - Make sure in Python don't use Debug mode. For example in FLASK.

         """
    if('gpt' in model.lower()):
        messages.append({"role": "system", "content":role})
        messages.append({"role": "user", "content":task})
        generated_code,messages=GPT(messages,lang,model)
    else:
        messages.append({"role": "system", "content":role})
        messages.append({"role": "user", "content":task})
        generated_code,messages=deployed_models(messages,lang,model)
        generated_code, messages=compilable(generated_code,lang)
    return generated_code, messages
        

def compilable(task,lang,model='gpt-3.5-turbo-0125'):
   role=f""" 
   - Make the given code in {lang} compilable, without any syntax errors on mac . If code is in C language it should be compilable using gcc. 
   - Only make code compilable, no other changes."""
   messages=[]
   messages.append({"role": "system", "content":role})
   messages.append({"role": "user", "content":task})
   generated_code,messages=GPT(messages,lang)
   return generated_code, messages  
    

# UtilityFunction For Evaluation

In [ ]:
# So, in the second loop, the messages should look like [system, user, assistant, user]
# In the third loop, it would be [system, user, assistant, user, assistant, user].. and so far
import os,re

# If vulnerability finder persist on presence of vulnerability then the number of vulnerable are extracted
def evaluator_score(text):
    # Regular expression pattern to find the desired substring followed by a number
    pattern = r"Score: (-?\d+)"
    
    # Search for the pattern in the provided text
    match = re.search(pattern, text)
    
    if match:
        #print(match)
        # Extract the number from the matched pattern
        return int(match.group(1))
    else:
        return 0
# Patcher score - updated score
def fixer_score(text):
    # Regular expression pattern to find the desired substring followed by a number
    pattern = r"d\) Updated SCORE: (-?\d+)"
    
    
    # Search for the pattern in the provided text
    match = re.search(pattern, text)
    
    if match:
        # Extract the number from the matched pattern
        return int(match.group(1))
    else:
        
        return 0  

import re

def found_vulnerabilities(text):
    # Regular expression pattern to find the CWE-IDs following the specified format
    pattern = r"D\.\s*CWE-IDs\s*of\s*found\s*vulnerabilities:\s*(.*)"
    
    # Search for the pattern in the provided text
    match = re.search(pattern, text, re.DOTALL | re.IGNORECASE)
    
    if match:
        # Return the captured CWE-IDs
        return match.group(1).strip()
    else:
        return "no vulnerability found"


def fixed_vulnerabilities(text):
    # Regular expression pattern to find the desired substring and capture CWE-IDs
   
    pattern=r"e\.\s*List\s*of\s*Fixed\s*Vulnerabilities:\s*(.*)"
    # Search for the pattern in the provided text
    match = re.search(pattern, text)
    
    if match:
        # Return the captured CWE-IDs
        return match.group(1).strip()
    else:
        return "no fixed vulnerabilities found"

# Utility function to extract the code from the fixed_code_format
# def extract_code(input_string):
#     # Regular expression pattern to find code blocks for C, C++, or Python
#     code_pattern = r"```(?:c|c\+\+|python)\n(.*?)```"
#     code_strings = re.findall(code_pattern, input_string, re.DOTALL | re.IGNORECASE)
    
#     # Join code blocks into a single string
#     code_string = '\n'.join(code_strings)
    
#     return code_string.strip()
def extract_code(input_string, lang):
    # Regular expression pattern to find code blocks for the specified language
    code_pattern = rf"```{re.escape(lang)}\n(.*?)```"
    code_strings = re.findall(code_pattern, input_string, re.DOTALL | re.IGNORECASE)
    
    # Join code blocks into a single string
    code_string = '\n'.join(code_strings)
    
    return code_string.strip()


            
def extract_fixed_vulnerabilities(input_text):
    search_text = "e) List of Fixed Vulnerabilities:"
    index = input_text.find(search_text)

    if index != -1:
        # Move index to the beginning of the substring
        index += len(search_text)

        # Extract substring starting from index
        result = input_text[index:].strip()

        return result
    else:
        return "Search text not found."


     


# Interactive iteration - Game Format Patcher patches and Vul_Finder detects Vulnerability, and update score

In [ ]:
def sec_code_for_loop(fixed_code,CWE_found,messages, index, score, model,lang,max_loop=10,temp=0):
    #previous_session_history = ''
    old_fixed_code=fixed_code
    #previous_vulnerability_report = "iteration 0 found vulnerability report: " + found_vulnerabilities(session_history)
    #previous_fixed_vulnerabilities = ''
    patched=0
    for i in range(max_loop):
#         print("Entered Loop ", i, "with vulnerable finder score ", score)
        newfixed_code=''
        tries=0
        while newfixed_code=='' and tries<5:
            fixed_code_format,  messages = patcher(messages,lang, score,model,temp)
            print(fixed_code_format)
            newfixed_code = extract_code(fixed_code_format,lang)
            print("tries ",tries)
            tries+=1
            
        fixed_code=newfixed_code
        fixed_score = fixer_score(fixed_code_format)
        list_fixed_vulnerabilities = extract_fixed_vulnerabilities(fixed_code_format)
        score = fixed_score
        
        if(fixed_code!=''):
            old_fixed_code=fixed_code
        else:
            fixed_code=old_fixed_code
       # print("############### FIXED CODE IS #####################\n",fixed_code_format)
        messages.append({"role": "assistant", "content":fixed_code})
        
        
        vulnerability_report, messages = vul_finder(messages,lang,model)
        #print("New vul Report ",vulnerability_report)
        messages.append({"role": "assistant", "content":vulnerability_report})
        CWE=extract_cwe_ids(vulnerability_report)
        maxKey=int(max(list(CWE_found.keys())))# Get old loop
        print("CWEs Found ",CWE_found.values())
        
        #print("new vul report\n",vulnerability_report)
        CWE_found[str(maxKey+1)]=CWE
        #print(f"In loop ${i + 1} of sec_for: ", messages)
        
       
        if "no vulnerabilities" in vulnerability_report.lower() or "vulnerable: no" in vulnerability_report.lower() or "n/a" in vulnerability_report.lower():
#             print("not vulnerable\n")
            #previous_session_history+="\n Result Fixed"
            patched=1
            model_fixed_code=fixed_code
            fixed_code, messages = compilable(fixed_code,lang)
            fixed_code=extract_code(fixed_code,lang)
            if(fixed_code==''):
                fixed_code=model_fixed_code
            return i+1, fixed_code, score,messages,patched
 
        else:
            eval_score = evaluator_score(vulnerability_report)
            new_report = found_vulnerabilities(vulnerability_report)
            #session_history ="In this code:\n"+fixed_code+"\n These security vulnerabilities were found:"+ new_report+ "previous history of vulnerabilities found include, "+previous_vulnerability_report+ "previous history of vulnerabilities fixed include, "+previous_fixed_vulnerabilities
            score = eval_score
    
    model_fixed_code=fixed_code
    fixed_code, messages = compilable(fixed_code,lang)
    fixed_code=extract_code(fixed_code,lang)
    if(fixed_code==''):
          fixed_code=model_fixed_code
    return i+1, fixed_code, score,messages,patched

## CodeQL Utilities 

In [ ]:
import subprocess
import os
import logging
from pathlib import Path

# Get the current working directory as a Path object
current_directory = Path.cwd()

# Print the current working directory
print("Current Directory:", current_directory)
def create_codeql_database(lang, source_root, output_dir,code_type):
    """Create a CodeQL database for the specified language and source directory.

    Args:
    lang (str): The programming language of the source code (e.g., 'cpp', 'java', 'python').
    source_root (str): The directory containing the source code to analyze.
    output_dir (str): The directory where the database should be stored.

    Returns:
    str: The path to the created database or None if creation failed.
    """
    print(source_root)
    # Define the database path
    db_path = os.path.join(output_dir, f"db-{lang}"+code_type)
    
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Prepare the CodeQL database creation command
    if(lang.lower()=='python'):
        command = [

            'codeql', 'database','create',db_path,
            '--language='+lang,
            '--source-root='+source_root,
            '--overwrite'

        ]
        
    else:
        print("Code is here")
        command = [

            'codeql', 'database','create',db_path,
            '--language=cpp',
            '--command=make',
            '--source-root='+source_root,
            '--overwrite'
        ]
    print(Path.cwd())
    print(' '.join(command))

    try:
        # Execute the command
        rm_command = ['rm', '-rf', db_path]
        subprocess.run(rm_command, check=True, capture_output=True)
        subprocess.run(command, check=True, capture_output=True)
        logging.info(f"Database created successfully at {db_path}.")
        return db_path
    except subprocess.CalledProcessError as e:
        logging.error("Failed to create database: %s", e.stderr.decode())
        return None

# Example Usage

def makedir(directory_path):
    try:
        os.mkdir(directory_path)
        print(f"Directory '{directory_path}' created successfully.")
    except FileExistsError:
        pass
    except Exception as e:
        pass

def save_code_snippet(codepath,code_snippet):
    """ Saves a code snippet to a file. """
    #print(code_snippet)
    with open(codepath, 'w') as file:
        file.write(code_snippet)
#     print('File Created ', codepath)
    
    return codepath
import os

def create_makefile(directory, output_file='Makefile'):
    # Ensure the path to the source files is correct
    source_directory = os.path.join(directory)
    
    # Collect all CPP files in the specified source directory
    cpp_files = [f for f in os.listdir(source_directory) if f.endswith('.c')]
    print(cpp_files)
    targets = [os.path.splitext(f)[0] for f in cpp_files]  # Extracting the base filename without extension
    
    # Start creating the Makefile content
    makefile_content = [
        "all: " + ' '.join(targets),
        ""
    ]

    # Add build rules for each target
    for target in targets:
        src_file = f"{target}.c"
        makefile_content.append(f"{target}: {src_file}")
        makefile_content.append(f"\tgcc {src_file} -o {target}")
        makefile_content.append("")

    # Add a clean rule to delete all compiled programs
    makefile_content.append("clean:")
    makefile_content.append(f"\trm -f {' '.join(targets)}")
    makefile_content.append("")
    makefile_content.append(".PHONY: all clean")

    # Write the content to a Makefile in the directory
    with open(os.path.join(directory, output_file), 'w') as file:
        file.write('\n'.join(makefile_content))

    print("Makefile created successfully.")


In [ ]:
import json
def run_queries(database_name,lang,filename):
#     print(database_name)
    querypath='./CodeQL/codeql-repo/'+lang+'/ql/src/top25/'
#     print(querypath)
    searchpath='./CodeQL/codeql-repo/'
    command = [

            'codeql', 'database','analyze',database_name, querypath, 
            '--format=sarif-latest',
            '--search-path='+searchpath,
            '--output='+database_name+'.sarif'

        ]
    try:
        subprocess.run(command, check=True, capture_output=True)
        logging.info(f"Queries analyzed successfully at {database_name}.")
#         print('Successfully analyzed the CodeQL database')
        f = open(database_name+'.sarif')
        result = json.load(f)
        result['status'] = 'success'
        result['type'] = lang
        with open(filename+'.json', 'wt', encoding='utf-8') as f:
            f.write(json.dumps(result))
            f.close()
    
    except subprocess.CalledProcessError as e:
        logging.error("Failed to  analyze: %s", e.stderr.decode())
    
        
                  
       

In [ ]:
import json
import csv

def extract_cwe_id(tags):
    """
    Extract CWE-ID from tags.
    """
    for tag in tags:
        if tag.startswith('external/cwe/cwe-'):
            return tag.split('-')[-1]
    return 'No CWE-ID available'

def process_json_file(json_file_path):
    # Load SARIF file
    with open(json_file_path, 'r') as file:
        sarif_data = json.load(file)

    # Dictionary to hold CWEs per file
    file_cwe_map = {}

    # Process each run in the SARIF file
    for run in sarif_data.get('runs', []):
        rules = {
            rule['id']: {
                'description': rule.get('shortDescription', {}).get('text', 'No description available'),
                'cwe': extract_cwe_id(rule.get('properties', {}).get('tags', []))
            } for rule in run.get('tool', {}).get('driver', {}).get('rules', [])
        }

        for result in run.get('results', []):
            rule_id = result.get('ruleId')
            message = result.get('message', {}).get('text', 'No message available')
            
            if result.get('locations'):
                location = result['locations'][0]
                uri = location['physicalLocation']['artifactLocation'].get('uri')
                region = location['physicalLocation'].get('region', {})
                start_line = region.get('startLine')
                start_column = region.get('startColumn')
                end_column = region.get('endColumn')
                
                location_str = f'line {start_line}, column {start_column}-{end_column}'

                # Get CWE-ID from the rules information
                cwe_id = rules[rule_id]['cwe']
                
                if uri not in file_cwe_map:
                    file_cwe_map[uri] = {'cwes': set(), 'messages': [], 'rule_ids': set(), 'locations': []}
                
                file_cwe_map[uri]['cwes'].add('CWE-'+cwe_id)  # Add the CWE-ID to the set for that file
                file_cwe_map[uri]['messages'].append(message)
                file_cwe_map[uri]['rule_ids'].add(rule_id)
                file_cwe_map[uri]['locations'].append(location_str)
                
    # Prepare the data to be returned
    summarized_data = []
    
    for filename, data in file_cwe_map.items():
        summarized_data.append({
            'filename': filename,
            'CWE': ', '.join(data['cwes']),
            'no of vul': len(data['cwes']),
            'rule': ', '.join(data['rule_ids']),
            'message': ' '.join(data['messages']),
            'locations': '; '.join(data['locations'])
        })

    return summarized_data

   

def json_to_csv(json_file_path):
    summarized_data = process_json_file(json_file_path)
    return summarized_data


def formatresult(summarized_data):
    if not summarized_data:
        Response = "A. Vulnerable: No\n"
        Response += "B. Score: 100\n"
        Response += "C. Vulnerabilities description: NO VULNERABILITIES\n"
        Response += "D. CWEs of found vulnerability: None"
    else:
        Response = "A. Vulnerable: Yes\n"
        no_of_vul = summarized_data[0]['no of vul']
        score = -1 * no_of_vul
        Response += f"B. Score: {score}\n"
        Response += "C. Vulnerabilities description:\n"
        Response += f"- Vulnerability Rule ID: {summarized_data[0]['rule']}\n"
        Response += f"- Vulnerability Message: {summarized_data[0]['message']}\n"
        Response += f"- Vulnerability CWEs: {summarized_data[0]['CWE']}\n"
        Response += f"- Line(s) of code: {summarized_data[0]['locations']}\n"
        Response += f"D. CWEs of found vulnerability: {summarized_data[0]['CWE']}"

    return Response
 
def extract_cwe_ids(text):
    # Regular expression to find CWE IDs with variations
    cwe_pattern = r'\bCWE-(\w{1,3})\b'
    
    # Find all CWE IDs in the text
    cwe_ids = re.findall(cwe_pattern, text, flags=re.IGNORECASE)
    
    # Deduplicate and return CWE IDs
    return format_cwe_ids(list(set(cwe_ids)))

def format_cwe_ids(cwe_ids):
    formatted_cwe_ids = []
    for cwe_id in cwe_ids:
        # Search for digits in the CWE ID
        match = re.search(r'\d+', cwe_id.lower())
        if match:
            # Extract and pad with leading zeros if necessary
            cwe_number = match.group().zfill(3)
            formatted_cwe_ids.append(f'CWE-{cwe_number}')
        else:
            # If no digits found, handle accordingly (e.g., log or skip)
            formatted_cwe_ids.append(cwe_id)  # Or handle as needed
    
    return formatted_cwe_ids
def check_cwe(cwe_list, cwe_to_check):
    return cwe_to_check in cwe_list 

In [ ]:
CODE_PATH='./Codes/Temp_Code/'
DATABASE_PATH='./Database/'
def vul_finder_codeQL(code_snippet, lang,code_type='_vul',path_folder='our'):
    
    
    if(lang=='c'):
        filepath=CODE_PATH+'c/'+path_folder+'/'
        makedir(filepath)
        codepath=filepath+'generated_code_a.c'
        save_code_snippet(codepath,code_snippet)
        create_makefile(filepath)
    else:
        filepath=CODE_PATH+'py/'+path_folder+'/'
        makedir(filepath)
        codepath=filepath+'generated_code_a.py'
        save_code_snippet(codepath,code_snippet)
        
    
    if(lang=='c'):
        db_path = create_codeql_database('cpp',filepath, DATABASE_PATH,code_type)
    else:
        db_path = create_codeql_database('python', filepath, DATABASE_PATH,code_type)
    
    if(db_path!=None):
        json_file_path=filepath+'/results_'+code_type
        if(lang=='c'):
                lang='cpp'
        try:
            run_queries(db_path,lang,json_file_path)
            summarized_data=json_to_csv(json_file_path+'.json')
            cleanup_files(codepath)
            cleanup_files(db_path)
            cleanup_files(json_file_path+'.json')
            return formatresult(summarized_data)
        except:
#             cleanup_files(codepath)
#             cleanup_files(db_path)
#             cleanup_files(json_file_path+'.json')
            return "Status -1"
    else:
#         cleanup_files(codepath)
#         cleanup_files(db_path)
#         cleanup_files(json_file_path+'.json')
        return "Status -100"

def cleanup_files(file_path):
    try:
        if os.path.exists(file_path):
            if os.path.isfile(file_path):
                os.remove(file_path)
                print("Done Removing ",file_path )
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
                print("Done Removing Folder ",file_path )
    except Exception as e:
        print(f"Failed to delete {file_path}: {e}")
   

In [ ]:
def extract_cwe_ids(text):
    cwe_pattern = r'\bCWE-(\w{1,3})\b'
    
    # Find all CWE IDs in the text
    cwe_ids = re.findall(cwe_pattern, text, flags=re.IGNORECASE)
    
    # Deduplicate and return CWE IDs
    return format_cwe_ids(list(set(cwe_ids)))

def format_cwe_ids(cwe_ids):
    formatted_cwe_ids = []
    for cwe_id in cwe_ids:
        # Search for digits in the CWE ID
        match = re.search(r'\d+', cwe_id.lower())
        if match:
            # Extract and pad with leading zeros if necessary
            cwe_number = match.group().zfill(3)
            formatted_cwe_ids.append(f'CWE-{cwe_number}')
        else:
            # If no digits found, handle accordingly (e.g., log or skip)
            formatted_cwe_ids.append(cwe_id)  # Or handle as needed
    
    return formatted_cwe_ids
def check_cwe(cwe_list, cwe_to_check):
    return cwe_to_check in cwe_list

import csv,re
import datetime

def read_c_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content


# RQ1

In [ ]:
# The pipeline
import csv
import datetime
#Change According to your Dataset

tries=10 # OurLoop Limit
limit=5# CodeQL Limit
datasets=['SecLLMHolmes.csv','LLMSecEval-prompts_.csv','SecureEvalDataset.csv']
modellist=['gpt-3.5-turbo-0125','gpt-4','gpt-4o','deepseek','llama']
savepath=''
datapath = './Dataset/baseline/'

for model in modellist:
    for dataset in datasets:
                datapath = datapath + dataset

                df = pd.read_csv(datapath)
                
                cql_database=f'GT_{model}_{dataset}'
                code_cql_folder=f'GT_{model}_{dataset}_'
                
                #Output file.csv
                with open(f"Pipeline/Proposed/Results/Final/RQ1/{model}_{dataset}.csv", "w+", newline='\n') as f:

                        writer = csv.writer(f)
                        writer.writerow(["index","NL Prompt","gt_CWE","language","original code","compilable_code","Detected_CWE","Correct_Detection","total_loop", "fixed code","decision_llm","llm_CWEs","Time (sec)","codeql_report","codeql_vul","decision_code_ql","codeql_org_vul","codeql_org_report","codeql_CWEs","codeql_loops","temperature"])  # Write header
                        for index, row in df.iterrows():
                            #try:
                              
                                    CWE_found={}
                                    CWE_found_CodeQL={}
                                    Originally_vulnerable=True
                                   
                                    
                                    print("################################### ROW ID ", index,"###################################")
                                    row_to_write=[]

                                    
                                    all_section_history=''
                                    if(dataset=='LLMSecEval-prompts_.csv'):
                                        lang=row['Language'].lower() 
                                        gt_CWE=row['Prompt ID'].split('_')[0]
                                        origin_code=row['original_code']
                                    elif(dataset=='SecureEvalDataset.csv'):
                                        lang='python'
                                        gt_CWE=row['CWE']
                                        origin_code=row['Original_Code']
                                    else:
                                        lang=row['Language'].lower() 
                                        gt_CWE=row['CWE']
                                        origin_code=row['Original_Code']
                                    
                                   
                                    
                                    
                                    
                                    generated_code,messages=compilable(origin_code,lang)
                                    original_code=extract_code(generated_code,lang)
                                    if(original_code==''):
                                        original_code=origin_code
                                    messages = []
                                    messages.append({"role": "assistant", "content": original_code})
                                    
                                    print("original_code\n ",original_code)
                                    print("generated_code\n ",generated_code)
                                    fixed_code=original_code       

                                    row_to_write.append(index)
                                    
                                    row_to_write.append(row['NL Prompt'])    
                                    row_to_write.append(gt_CWE)
                                    row_to_write.append(lang)
                                    row_to_write.append(origin_code)
                                    row_to_write.append(generated_code)
                                    # Given NLP prompt generate a code
                                    start_time = datetime.datetime.now()
                                    decision_our=''

                                    vulnerability_report, messages=vul_finder(messages,lang,model)
                                    messages.append({"role": "assistant", "content":vulnerability_report})
                                    print("\nAfter vul_finder code: ", vulnerability_report)
                                    CWE=extract_cwe_ids(vulnerability_report)
                                    #END VUL REPORT
                                    CWE_found['0']=CWE
                                    print(f"GT CWE {gt_CWE}, found CWE {CWE}")
                                    if(CWE !=None):
                                        row_to_write.append(CWE)
                                        if (gt_CWE) in CWE:
                                            row_to_write.append('Yes')
                                        else:
                                            row_to_write.append('No')
                                    else:
                                        row_to_write.append('N/A')
                                        row_to_write.append('N/A')
                                    #print("Our Report ",vulnerability_report)
                                    score=evaluator_score(vulnerability_report)
                                    print("Current Score ",score)
                                    decision_our=''
                                    if "no vulnerabilities" in vulnerability_report.lower() or "vulnerable: no" in vulnerability_report.lower() or "n/a" in vulnerability_report.lower(): 
                                        loop_count=0
                                        patched=1
                                        Originally_vulnerable=False

                                        decision_our="not vulnerable"
                                    else:
                                        # all_section_history+="\n"+session_history
                                        loop_count, fixed_code, score,messages,patched=sec_code_for_loop(fixed_code,CWE_found,messages,index,score,model,lang,tries)
                                        # Simulate a process by adding a delay (e.g., using sleep)

                                        if(patched==0):
                                            decision_our="not fixed"
                                        else:
                                            decision_our="fixed"


                                    #################################################fixed_code################################################
                                    #################################################CODEQL STARTS HERE ##############
                                    codeql_vul='Yes'
                                    limit=10
                                    temp=0
                                    additional_tries=0
                                    decision_codeql=''
                                    #print(decision_our == "not vulnerable")
                                    while codeql_vul.lower()=='yes' and temp<=0: # give 10 extra tries
                                        previous_loop_count=loop_count
                                        codeql_report= vul_finder_codeQL(fixed_code, lang,cql_database,code_cql_folder)
                                        print("\nCode QL vul_finder: ", codeql_report)
                                        if('status' not in codeql_report.lower()):
                                            CWE=extract_cwe_ids(codeql_report)

                                            #END VUL REPORT
                                            CWE_found_CodeQL[str(additional_tries+1)]=CWE

                                        if "no vulnerabilities" in codeql_report.lower() or "vulnerable: no" in codeql_report.lower():
                                                codeql_vul='No'
                                                decision_codeql="not vulnerable"
                                                break

                                        else:
                                            if("status" in codeql_report.lower()):
                                                codeql_vul='Error'
                                                decision_codeql="can't compile"
                                                message="The code is not compilable, complete it and make it runnable without errors , make sure all required header files are included. Correct the brackets and identation\n"
                                            else:
                                                message="The code is still vulnerable. Here is the CodeQL report\n"
                                            patched=0
                                            score=evaluator_score(codeql_report)
                                            messages.append({"role": "user", "content":fixed_code})
                                            messages.append({"role": "assistant", "content":message+codeql_report})
                                            if(additional_tries>=limit):
                                                temp+=0.1
                                                break
                                            loop_count, fixed_code, score,messages,patched=sec_code_for_loop(fixed_code,CWE_found,messages,index,score,model,lang,tries,temp)

                                            # Simulate a process by adding a delay (e.g., using sleep)
                                            loop_count+=previous_loop_count
                                            additional_tries+=1
                                            # Determine decision_our based on patch status and codeql findings
                                            if patched == 0 :
                                                decision_our += ", Remains not fixed by llm even after CodeQL"
                                            elif patched==1:
                                                decision_our+= ", Fixed by llm after CodeQL"


                                    if(codeql_vul=='Yes'):
                                            codeql_report= vul_finder_codeQL(fixed_code, lang,cql_database,code_cql_folder)
                                            if('status' not in codeql_report.lower()):
                                                CWE=extract_cwe_ids(codeql_report)

                                                #END VUL REPORT
                                                CWE_found_CodeQL[str(additional_tries+1)]=CWE

                                            if "no vulnerabilities" in codeql_report.lower() or "vulnerable: no" in codeql_report.lower():
                                                    codeql_vul='No'
                                                    decision_codeql+=", not vulnerable after"+str(limit)+"tries"
                                            elif("status" in codeql_report.lower()):
                                                    codeql_vul='Error'
                                                    decision_codeql+=", can't compile after"+str(limit)+"tries"
                                            else:
                                                score=evaluator_score(codeql_report)

                                                messages.append({"role": "user", "content":fixed_code})
                                                messages.append({"role": "assistant", "content":codeql_report})
                                                decision_codeql+=", still vulnerable according to CodeQL"
                                                print("Still Vulnerable after CODEQL as well")



                                    end_time = datetime.datetime.now()
                                    elapsed_time = (end_time - start_time).total_seconds()

                                    #print("codeql_result ",codeql_report)

                                    codeql_org_vul='Yes'
                                    codeql_org_report= vul_finder_codeQL(original_code, lang,cql_database,code_cql_folder)
                                    if('status' not in codeql_org_report.lower()):
                                        CWE=extract_cwe_ids(codeql_org_report)
                                                #END VUL REPORT
                                        CWE_found_CodeQL[0]=CWE

                                    if "no vulnerabilities" in codeql_org_report.lower() or "vulnerable: no" in codeql_org_report.lower():
                                                    codeql_org_vul='No'
                                    elif("status" in codeql_org_report.lower()):
                                                    codeql_org_vul='Error'
                                    else:
                                            print("Org was Vulnerable")
                                    add_row=[loop_count, fixed_code, decision_our,CWE_found,elapsed_time,codeql_report,codeql_vul,decision_codeql,codeql_org_vul,codeql_org_report,CWE_found_CodeQL,additional_tries,temp]

                        #             # Add detailed logging for debugging and traceability
                                    print(f"Loop Count: {loop_count}")
                                    print(f"Decision Original CodeQL Vulnerable: {codeql_org_vul}")
                                    print(f"Decision CodeQL: {decision_codeql}")
                                    print(f"Decision Our: {decision_our}")
                                    #Update Based on your Folder
                                    messages_str = '. '.join([f"{d['role']}: {d['content']}" for d in messages])

                                    
                                    row_to_write.extend(add_row)
                                    writer.writerow(row_to_write)
#                             except:
#                                     pass

# RQ 2 on wards

In [ ]:
# The pipeline
import csv
import datetime
datasets=['SecLLMHolmes.csv','LLMSecEval-prompts_.csv','SecureEvalDataset.csv']
modellist=['gpt-3.5-turbo-0125','gpt-4','gpt-4o','deepseek','llama']
savepath='Result folder path'
for dataset in datasets:
    for model in modellist:
        
                cql_database=f'Fin_Enc_role_COC_{model}_{dataset}'
                code_cql_folder=f'Fin_Enc_role_COC_{model}_{dataset}'
                
                df = pd.read_csv('dataset path'+dataset)

                #Output file.csv
                with open(f"{savepath}/{model}_{dataset}", "w+", newline='\n') as f:

                        writer = csv.writer(f)
                        writer.writerow(["index","nlp prompt","gt_CWE","language","generated code","total_loop", "fixed code","decision_llm","llm_CWEs","Time (sec)","codeql_report","codeql_vul","decision_code_ql","codeql_org_vul","codeql_org_report","codeql_CWEs","codeql_loops","temperature"])  # Write header

                        for index, row in df.iterrows():
                            
                                messages = []
                                CWE_found={}
                                CWE_found_CodeQL={}
                                Originally_vulnerable=True
                                print("################################### Row ID ", index,"###################################")
                                row_to_write=[]

                                row_to_write.append(index)
                                all_section_history=''
                                lang=row['language'].lower() 
                                gt_CWE=row['gt_CWE']
                                print("GroundTruth CWE: ", gt_CWE)

                                task=row['nlp prompt']
                                generated_code,messages=generator(task,messages,lang,model)
                                messages.append({"role": "assistant", "content": generated_code})
                                original_code=extract_code(generated_code,lang)
                                       
                             
                                row_to_write.append(task)
                                row_to_write.append(gt_CWE)
                                row_to_write.append(lang)
                                # Given NLP prompt generate a code
                                start_time = datetime.datetime.now()
                                decision_our=''
                               
                                vulnerability_report, messages=vul_finder(messages,lang,model)
                                messages.append({"role": "assistant", "content":vulnerability_report})
                                #print("\n\n\nAfter vul_finder code: ", vulnerability_report)
                                CWE=extract_cwe_ids(vulnerability_report)
                                #END VUL REPORT
                                CWE_found['0']=CWE

                                #print("Our Report ",vulnerability_report)
                                score=evaluator_score(vulnerability_report)
                                print("Current Score ",score)
                                decision_our=''
                                if "no vulnerabilities" in vulnerability_report.lower() or "vulnerable: no" in vulnerability_report.lower() or "n/a" in vulnerability_report.lower(): 
                                    loop_count=0
                                    patched=1
                                    Originally_vulnerable=False
                                    fixed_code=original_code
                                    decision_our="not vulnerable"
                                else:
                                    # all_section_history+="\n"+session_history
                                    loop_count, fixed_code, score,messages,patched=sec_code_for_loop(CWE_found,messages,index,score,model,lang,tries)
                                    # Simulate a process by adding a delay (e.g., using sleep)

                                    if(patched==0):
                                        decision_our="not fixed"
                                    else:
                                        decision_our="fixed"


                                #################################################fixed_code################################################
                                #################################################CODEQL STARTS HERE ##############
                                codeql_vul='Yes'
                                limit=13
                                temp=0
                                additional_tries=0
                                decision_codeql=''
                                #print(decision_our == "not vulnerable")
                                while codeql_vul=='Yes' and temp<1: # give 10 extra tries
                                    previous_loop_count=loop_count
                                    codeql_report= vul_finder_codeQL(fixed_code, lang,cql_database,code_cql_folder)
                                    if('status' not in codeql_report.lower()):
                                        CWE=extract_cwe_ids(codeql_report)

                                        #END VUL REPORT
                                        CWE_found_CodeQL[str(additional_tries+1)]=CWE

                                    if "no vulnerabilities" in codeql_report.lower() or "vulnerable: no" in codeql_report.lower():
                                            codeql_vul='No'
                                            decision_codeql="not vulnerable"
                                            break

                                    else:
                                        if("status" in codeql_report.lower()):
                                            codeql_vul='Error'
                                            decision_codeql="can't compile"
                                            message="The code is not compilable, make sure all required header files are included.\n"
                                        else:
                                            message="The code is still vulnerable. Here is the CodeQL report\n"
                                        patched=0
                                        score=evaluator_score(codeql_report)
                                        messages.append({"role": "user", "content":fixed_code})
                                        messages.append({"role": "assistant", "content":message+codeql_report})
                                        if(additional_tries>=limit):
                                            temp+=0.1
                                            
                                        loop_count, fixed_code, score,messages,patched=sec_code_for_loop(CWE_found,messages,index,score,model,lang,tries,temp)
                                        
                                        # Simulate a process by adding a delay (e.g., using sleep)
                                        loop_count+=previous_loop_count
                                        additional_tries+=1
                                        # Determine decision_our based on patch status and codeql findings
                                        if patched == 0 :
                                            decision_our += ", Remains not fixed by llm even after CodeQL"
                                        elif patched==1:
                                            decision_our+= ", Fixed by llm after CodeQL"


                                if(codeql_vul=='Yes'):
                                        codeql_report= vul_finder_codeQL(fixed_code, lang,cql_database,code_cql_folder)
                                        if('status' not in codeql_report.lower()):
                                            CWE=extract_cwe_ids(codeql_report)

                                            #END VUL REPORT
                                            CWE_found_CodeQL[str(additional_tries+1)]=CWE

                                        if "no vulnerabilities" in codeql_report.lower() or "vulnerable: no" in codeql_report.lower():
                                                codeql_vul='No'
                                                decision_codeql+=", not vulnerable after"+str(limit)+"tries"
                                        elif("status" in codeql_report.lower()):
                                                codeql_vul='Error'
                                                decision_codeql+=", can't compile after"+str(limit)+"tries"
                                        else:
                                            score=evaluator_score(codeql_report)

                                            messages.append({"role": "user", "content":fixed_code})
                                            messages.append({"role": "assistant", "content":codeql_report})
                                            decision_codeql+=", still vulnerable according to CodeQL"
                                            print("Still Vulnerable after CODEQL as well")
                                       


                                end_time = datetime.datetime.now()
                                elapsed_time = (end_time - start_time).total_seconds()

                                
                                codeql_org_vul='Yes'
                                codeql_org_report= vul_finder_codeQL(original_code, lang,cql_database,code_cql_folder)
                                if('status' not in codeql_org_report.lower()):
                                    CWE=extract_cwe_ids(codeql_org_report)
                                            #END VUL REPORT
                                    CWE_found_CodeQL[0]=CWE

                                if "no vulnerabilities" in codeql_org_report.lower() or "vulnerable: no" in codeql_org_report.lower():
                                                codeql_org_vul='No'
                                elif("status" in codeql_org_report.lower()):
                                                codeql_org_vul='Error'
                                else:
                                        print("Org was Vulnerable")
                                add_row=[original_code,loop_count, fixed_code, decision_our,CWE_found,elapsed_time,codeql_report,codeql_vul,decision_codeql,codeql_org_vul,codeql_org_report,CWE_found_CodeQL,additional_tries,temp]

                    #             # Add detailed logging for debugging and traceability
                                print(f"Loop Count: {loop_count}")
                                print(f"Decision Original CodeQL Vulnerable: {codeql_org_vul}")
                                print(f"Decision CodeQL: {decision_codeql}")
                                print(f"Decision Our: {decision_our}")
                                #Update Based on your Folder
                                messages_str = '. '.join([f"{d['role']}: {d['content']}" for d in messages])

                                with open(f"./session_history/Final_ECOC_{index}_{model}_{dataset}.txt", "w") as f:
                                      f.write(messages_str)

                                row_to_write.extend(add_row)
                                writer.writerow(row_to_write)
                #             except:
                #                 pass